After finishing building spe dataset, next step is building a dataset for bulk measurements.

In [1]:
import numpy as np 
import pandas as pd
import glob
import matplotlib.pyplot as plt

#plt.style.use('ggplot')
plt.style.use('seaborn-colorblind')
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'

%matplotlib inline

import datetime
date = datetime.datetime.now().strftime('%Y%m%d')

# Read the bulk measurements

In [2]:
glob.glob('data/Bulk chem/*')

['data/Bulk chem/SO264-64-1_bulk.xlsx',
 'data/Bulk chem/SO264-66-2_bulk.xlsx',
 'data/Bulk chem/SO264-56-2_bulk.xlsx',
 'data/Bulk chem/SO264-28-2_bulk.xlsx',
 'data/Bulk chem/SO264-55-1_bulk.xlsx',
 'data/Bulk chem/SO264-15-2_bulk.xlsx',
 'data/Bulk chem/SO264-13-2_bulk.xlsx']

In [14]:
bulk_df = pd.DataFrame()
for core in glob.glob('data/Bulk chem/*'):
    X = pd.read_excel(core, sheet_name = 1, usecols = ['mid depth(cm)', 'TC%', 'TOC%', 'CaCO3%'])
    X['core'] = [core.split('/')[-1][:-10] for _ in X.index]
    bulk_df = pd.concat([bulk_df, X], axis = 0, join = 'outer')

bulk_df

,mid depth(cm),TC%,TOC%,CaCO3%,core
0,11.5,2.542079,0.394127,17.898887,SO264-64-1
1,21.5,2.247150,0.611208,13.632300,SO264-64-1
2,30.5,0.710588,0.523402,1.559822,SO264-64-1
3,101.5,0.562171,0.472551,0.746802,SO264-64-1
4,181.5,0.578167,0.312852,2.210866,SO264-64-1
...,...,...,...,...,...
34,1143.5,6.221641,0.269061,49.602848,SO264-13-2
35,1153.5,7.961034,0.254319,64.220052,SO264-13-2
36,1183.5,6.598575,0.175505,53.523438,SO264-13-2
37,1193.5,8.530931,0.186714,69.532368,SO264-13-2


In [26]:
#bulk_df['mid_depth_mm'] = bulk_df['mid depth(cm)'] * 10
#bulk_df.drop('mid depth(cm)', axis = 1, inplace = True)
bulk_df = bulk_df.astype({'mid_depth_mm': 'int'})
bulk_df

,TC%,TOC%,CaCO3%,core,mid_depth_mm
0,2.542079,0.394127,17.898887,SO264-64-1,115
1,2.247150,0.611208,13.632300,SO264-64-1,215
2,0.710588,0.523402,1.559822,SO264-64-1,305
3,0.562171,0.472551,0.746802,SO264-64-1,1015
4,0.578167,0.312852,2.210866,SO264-64-1,1815
...,...,...,...,...,...
383,6.221641,0.269061,49.602848,SO264-13-2,11435
384,7.961034,0.254319,64.220052,SO264-13-2,11535
385,6.598575,0.175505,53.523438,SO264-13-2,11835
386,8.530931,0.186714,69.532368,SO264-13-2,11935


## Export dataset

In [28]:
bulk_df = bulk_df.reset_index(drop = True)
bulk_df.to_csv('data/bulk_dataset_{}.csv'.format(date))

# Merge spe and bulk datasets

In [41]:
spe_df = pd.read_csv('data/spe_dataset_20201007.csv', index_col = 0)

In [42]:
mask_c = spe_df.columns[:2048]  # only the channels
merge_df = pd.DataFrame()

for index, row in bulk_df.iterrows():
    mid = row['mid_depth_mm']
    core = row['core']
    
    # get the spe in 10 mm interval
    mask_r = (spe_df.composite_depth_mm >= (mid-5)) & (spe_df.composite_depth_mm <= (mid+5)) & (spe_df.core == core)
    merge_df = pd.concat(
        [merge_df, spe_df.loc[mask_r, mask_c].apply(np.mean, axis = 0).append(row)],
        axis = 1
    )
    
merge_df = merge_df.T.reset_index(drop = True)

In [33]:
merge_df

,0,1,2,3,4,5,6,7,8,9,...,2043,2044,2045,2046,2047,TC%,TOC%,CaCO3%,core,mid_depth_mm
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2.54208,0.394127,17.8989,SO264-64-1,115
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2.24715,0.611208,13.6323,SO264-64-1,215
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.710588,0.523402,1.55982,SO264-64-1,305
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.562171,0.472551,0.746802,SO264-64-1,1015
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.578167,0.312852,2.21087,SO264-64-1,1815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,6.22164,0.269061,49.6028,SO264-13-2,11435
384,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,7.96103,0.254319,64.2201,SO264-13-2,11535
385,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,6.59857,0.175505,53.5234,SO264-13-2,11835
386,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,8.53093,0.186714,69.5324,SO264-13-2,11935


In [43]:
merge_df[merge_df.isnull().any(axis = 1)]

,0,1,2,3,4,5,6,7,8,9,...,2043,2044,2045,2046,2047,TC%,TOC%,CaCO3%,core,mid_depth_mm
39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.300857,0.275425,0.211924,SO264-66-2,795
40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.299891,0.229273,0.588459,SO264-66-2,995
149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,9.91158,0.151331,81.3321,SO264-28-2,7595
150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7.79372,0.154413,63.6583,SO264-55-1,25
160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.6566,0.649003,25.0623,SO264-55-1,975
170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.7369,0.422641,10.9517,SO264-55-1,2045
239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.28527,0.267357,8.48227,SO264-55-1,9035
308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.32614,0.224245,0.849089,SO264-55-1,16035
330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,8.79912,0.123676,72.2924,SO264-15-2,7805
331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.9751,0.0985913,90.6339,SO264-15-2,8005


In [45]:
np.unique(merge_df.loc[merge_df.isnull().any(axis = 1), 'core'])

array(['SO264-15-2', 'SO264-28-2', 'SO264-55-1', 'SO264-66-2'],
      dtype=object)

In [39]:
spe_df[spe_df.core == 'SO264-55-1'].to_csv('results/check.csv')

Hmm...It's weird. After checking the spe files, I found:<br>
SO264-55-1: the scanning depth of the section Rescan00050 starts at 70.0 mm instead of 0.0 or 10.0 mm. JC confirmed that it's because before 70 mm, the sediments are too soft to be scanned. This cause the shift of the composite depth because the section depth were set to start at 0.<br>
Since I found SO264-66-2 section also has this problem, I discuss with JC and decide to redo the workflow from the begining to simply sum up the section start depth and scanning depth to be the composite depth, like we did in the pilot_test.ipynb. Ignore the 0.0 or 10.0 mm problem because it's just 1 cm deviation at most.